# Oppgave 7

In [5]:
import sqlite3
import pandas as pd
from tabulate import tabulate
import os

def get_file_name_of_newest_database():
    base_name = "database"
    counter = 1

    # Increment counter until a new file name is found
    while os.path.exists(f"{base_name}{counter}.db"):
        counter += 1
    counter -= 1
    return f"{base_name}{counter}.db"

name = get_file_name_of_newest_database()
conn = sqlite3.connect(name)
cursor = conn.cursor()

In [6]:
# Du skal lage et Pythonprogram (og SQL) som tar et skuespillernavn og finner
# hvilke skuespilllere de har spilt med i samme akt. Skriv ut navn på begge og
# hvilket skuespill det skjedde.
# jeg anter at det menes act og teaterstykke når det står skuespill

# finn alle akter som skuespilleren har spilt i
def hent_akter(skuespiller):
    query = """
    SELECT DISTINCT RPA.Actnummer, S1.TeaterStykkeNavn
    FROM Skuespiller S1
    JOIN RollePaAkt RPA ON S1.TeaterStykkeNavn = RPA.teaterstykkeNavn AND S1.RolleNavn = RPA.rolleNavn
    JOIN Ansatt A1 on S1.Identifikator = A1.ansattId
    WHERE A1.navn = ?
    """
    cursor.execute(query, (skuespiller,))
    akter = cursor.fetchall()
    return akter


def hent_skuespillere_i_samme_akt(skuespiller):
    akter = hent_akter(skuespiller)
    print(tabulate
          (akter, headers= ["Akt", "Teaterstykke"], tablefmt="grid"))
    query = """
    SELECT
        S1.teaterstykkenavn,
        S2.navn as Actor,
        S2.RolleNavn,
        A1.navn as CoActor,
        S1.rolleNavn,
        RPA.Actnummer
    FROM
        Skuespiller S1
        join Ansatt A1 on S1.Identifikator = A1.ansattId
        JOIN RollePaAkt RPA ON S1.TeaterStykkeNavn = RPA.teaterstykkeNavn AND S1.RolleNavn = RPA.rolleNavn
        JOIN (
            SELECT S2.TeaterStykkeNavn, S2.RolleNavn, A2.navn, RPA2.Actnummer, A2.AnsattId
            FROM Skuespiller S2
            JOIN Ansatt A2 on S2.Identifikator = A2.ansattId
            JOIN RollePaAkt RPA2 ON S2.TeaterStykkeNavn = RPA2.teaterstykkeNavn AND S2.RolleNavn = RPA2.rolleNavn
            WHERE A2.navn = ?
        ) as S2
        ON S1.TeaterStykkeNavn = S2.TeaterStykkeNavn AND RPA.Actnummer = S2.Actnummer And A1.AnsattId != S2.AnsattId
        order by S1.rolleNavn

    """
    # cursor.execute(query, (skuespiller,))
    cursor.execute(query, (skuespiller,))
    
    # print column names
    print()
    skuespillere = cursor.fetchall()
    return skuespillere

# get all actors names 
query = "SELECT navn FROM Ansatt"
cursor.execute(query)
actors = cursor.fetchall()

# skuespillere = hent_skuespillere_i_samme_akt("Thomas Jensen Takyi")
skuespillere = hent_skuespillere_i_samme_akt("Ingunn Beate Strige Øyen")
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––")

headers = [description[0] for description in cursor.description]
print(tabulate(skuespillere, headers=headers, tablefmt="grid"))
# print(tabulate(skuespillere, headers=["Skuespiller 1", "Skuespiller 2", "Teaterstykke", "Akt"], tablefmt="grid"))

+-------+----------------+
|   Akt | Teaterstykke   |
+=======+================+
|     1 | Kongsemnene    |
+-------+----------------+
|     3 | Kongsemnene    |
+-------+----------------+

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
+--------------------+--------------------------+------------------+----------------------------+-------------------+-------------+
| TeaterStykkeNavn   | Actor                    | RolleNavn        | CoActor                    | RolleNavn         |   Actnummer |
+====================+==========================+==================+============================+===================+=============+
| Kongsemnene        | Ingunn Beate Strige Øyen | Inga fra Vartejg | Thomas Jensen Takyi        | Biskop Nikolas    |           1 |
+--------------------+--------------------------+------------------+----------------------------+-------------------+-------------+
| Kongsemnene        | Ingunn Beate Strige Øyen | Inga fra Vartejg | Thomas J